In [1]:
import pandas as pd
import json
import time
from datetime import timedelta, date, datetime 
import altair as alt
import numpy as np

KeyboardInterrupt: 

In [24]:
trial_series = np.random.rand(10)
len(trial_series)


10

In [28]:
for i in range(0,10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [31]:

def moving_averages(series, window):
    new_series = []
    for i in (0, len(series)):
      if i < (window):
          new_series.append(pd.NA)
      else:
          elms_in_window = []
          for j in (0,window):
              elms_in_window.append(series[i-window+1+j])
          new_series.append(np.mean(elms_in_window))    

        #   element_in_window = [series[i - window + 1], series[i-wi]]      

    return new_series

In [32]:
moving_averages(trial_series,3)

IndexError: index 11 is out of bounds for axis 0 with size 10

In [3]:
PATH = 'all_incidents.json'

def load_data():
    with open(PATH, 'r') as json_file:
        data = json.load(json_file)
    return pd.DataFrame(data)

def format_datetime(data):
    data['date'] = pd.to_datetime(data['date'], format = "%d %b %Y" )


    data['start_time_obj'] = pd.to_datetime(data['start_time'], format = "%H:%M")
    data['end_time_obj'] = pd.to_datetime(data['end_time'], format = "%H:%M")

    # to add one day delta to the end time if it ends in the next day
    data['next_day'] = data['end_time_obj'] < data['start_time_obj']   
    data.loc[data['next_day'] == True,'end_time_obj'] =  data['end_time_obj'] +  pd.Timedelta(days=1)


    # calculate time used
    data['time_used'] = data['end_time_obj'] - data['start_time_obj']
    # data['time_used'] = data['time_used'].total_seconds()
    data['time_used'] = data['time_used'].apply(pd.Timedelta.total_seconds)
    # turn seconds to hours
    data['time_used'] = data['time_used']/3600

    # convert start time and end time to float and based on from '1900-01-01T00:00:00'
    base_datetime = datetime.strptime('01 Jan 1900', "%d %b %Y")
    data['start'] = data['start_time_obj'] - base_datetime
    data['start'] = (data['start'].apply(pd.Timedelta.total_seconds))/3600
    data['end'] = data['end_time_obj'] - base_datetime
    data['end'] = (data['end'].apply(pd.Timedelta.total_seconds))/3600

    return data

In [ ]:
data = load_data()
data = format_datetime(data)


In [5]:
data.describe()

,date,start_time_obj,end_time_obj,time_used,start,end
count,2077,2555,2003,2003.000000,2555.000000,2003.000000
mean,2015-09-07 06:55:17.477130496,1900-01-01 15:44:44.524462080,1900-01-01 19:36:31.812281600,3.879206,15.745701,19.608837
min,1968-10-07 00:00:00,1900-01-01 00:00:00,1900-01-01 00:30:00,0.050000,0.000000,0.500000
25%,2012-06-30 00:00:00,1900-01-01 13:39:30,1900-01-01 17:00:00,1.716667,13.658333,17.000000
50%,2017-02-26 00:00:00,1900-01-01 16:29:00,1900-01-01 19:37:00,3.333333,16.483333,19.616667
75%,2021-10-17 00:00:00,1900-01-01 19:00:30,1900-01-01 22:31:00,5.158333,19.008333,22.516667
max,2025-03-24 00:00:00,1900-01-01 23:58:00,1900-01-02 19:04:00,23.516667,23.966667,43.066667
std,NaN,NaN,NaN,3.141492,4.997613,5.584496


In [6]:
data = data[data['date'] > pd.Timestamp(2012, 1, 1, 0) ]


In [7]:
base = alt.Chart(data).encode(
    x = alt.X('yearmonth(date):T')).configure_view(
    continuousWidth=1200,
    )

In [8]:
line = base.mark_line().encode(
    y = alt.Y('count():Q'),
    tooltip=alt.Tooltip(['yearmonth(date):T','count():Q'])
    )
line

alt.Chart(...)

In [9]:
tick = base.mark_tick().encode(
    y='mean(count())',
)
tick

ValueError: Unable to determine data type for the field "mean(count())"; verify that the field name is not misspelled. If you are referencing a field from a transform, also confirm that the data type is specified correctly.

alt.Chart(...)